In [2]:
import derinet.lexicon as dlex
import os
lexicon = dlex.Lexicon()
current_dir = os.getcwd()  # aktualni adresar
file_path = os.path.join(current_dir, "./derinet-2-3.tsv")  #sestaveni cesty
lexicon.load(file_path)

all_lemmas = {lex.lemma for lex in lexicon.iter_lexemes()}

    test 8: kontrola přiřazení číslovek podle koncovky (-tý, -tero, -terý)

In [28]:
def tvorba_predku():
    predci = set()
    for lex in lexicon.iter_lexemes(): 
        if lex.pos == "NUM":
            predci.add(lex.lemma)
    return predci 

def hledani_predku(predci, dite):
    mozni = set()
    for lex in predci:
        if lex in dite and lex != dite: 
            mozni.add(lex)
    
    konecny = set()
    #tohle to míří na to, abych tam neměl sto a stodva zároveň
    for i in mozni:
        for j in mozni:
            if i in j and i != j:
                konecny.add(i)
    
    return mozni.difference(konecny)

def vypisovani(existuji, neexistuji):
    with open("test8.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
        f.write("ČÍSLOVKY KONČÍCÍ NA 'ERO', 'ERÝ', 'KRÁT', 'ISTÝ', KTERÉ JSOU BEZ RODIČE \n")
        f.write("ČÍSLOVKY S NALEZENÝMI PŘEDKY: \n")

        f.write("PŘÍKLAD TISKU: \n")
        f.write("číslovka \n")
        f.write(f"  předek\n")
        f.write("\n")

        for pod, cisla in sorted(existuji, key=lambda x: len(x[1]), reverse=False):
            f.write(f"{pod}\n")
            for i in cisla:
                f.write(f"  {i}\n")

        f.write("\n")
        f.write("ČÍSLOVKY, PRO KTERÉ NEBYL NALEZEN PŘEDEK: \n")
        f.write("\n")
        for i in neexistuji:
            f.write(f"{i} \n")

#mireno na slova jako desatero, devátý,

seznam = []
bez = []
predci = tvorba_predku()

for lexeme in lexicon.iter_lexemes():
    if lexeme.pos == "NUM" and lexeme.lemma.endswith(("tero", "terý", "krát", "istý", "erý", "ero")) :
        if lexeme.all_parents == []:
            moznosti = []
            hotovo = False
            rodic = None

            if lexeme.lemma.endswith("krát"):
                rodic = lexeme.lemma[:-4]
            elif lexeme.lemma.endswith("istý"):
                moznosti.append("sto")
                rodic = lexeme.lemma[:-4]
            elif lexeme.lemma.endswith(("erý", "ero")):
                rodic = lexeme.lemma[:-3]
            else:
                print(lexeme.lemma)

            #když stačí uříznout jenom konec
            if rodic is not None:
                if rodic in all_lemmas:
                    moznosti.append(rodic)
                    hotovo = True
            
            if not hotovo:  
                dalsi = None      
                if rodic.endswith(("dvacát", "čtyřicát", "třicat")):
                    dalsi = rodic[:-2] + "e" + rodic[-1]
                elif rodic.endswith("devat"):
                    dalsi = rodic[:-2] + "ě" + rodic[-1]
                elif rodic == "desat":
                    dalsi = "deset"
                elif rodic.endswith("desat"):
                    dalsi = rodic[:-2] + "á" + rodic[-1]
                elif rodic.startswith("dvou"):
                    dalsi = rodic[4:]
                    moznosti.append("dva")
                elif rodic.startswith("tří"):
                    dalsi = rodic[3:]
                    moznosti.append("tři")
                elif lexeme.lemma.endswith("krát"):
                    dalsi = lexeme.lemma[:-3]
                elif rodic == "st":
                    moznosti.append("sto")
                    hotovo = True

                if dalsi is not None:
                    if dalsi in all_lemmas:
                        moznosti.append(dalsi)
                        hotovo = True
                    else:
                        rodic = dalsi
                
                if not hotovo:
                    mozni_rodice = list(hledani_predku(predci, rodic))
                    if mozni_rodice != []:
                        moznosti_pom = moznosti + mozni_rodice
                        moznosti = moznosti_pom
                        hotovo = True

                    else:
                        bez.append(lexeme.lemma)
            
            if moznosti != []:
                seznam.append((lexeme.lemma, moznosti))
             
                    
vypisovani(seznam, bez)


test 9: když podstatné jméno ženského rodu končí na 'ka', tak je to odvozené od mužského (ne všechny, ale spoustu z nich -> asistentka x matka)

In [ ]:
def nejlepsi(kandidati):
    hodnota = 0
    nejvyssi = None 
    for i in kandidati:
        #ještě bych sem mohla přidat kontrolu, že je to rodu mužského, ale to nevím, jestli by zvládl derinet
        #jestli bych se nepřipravil o to slovo
        if i != None:
            slovo = lexicon.get_lexemes(i)[0]
            if slovo.misc['corpus_stats'] and slovo.misc['corpus_stats']['relative_frequency'] > hodnota:
                nejvyssi = slovo.lemma
                hodnota = slovo.misc['corpus_stats']['relative_frequency']    

    return nejvyssi 

def test(nove):
    kandidati = set()
    for i in "aeioyu":
        if nove + i in all_lemmas:
            slovicko = nove + i
            if slovicko in all_lemmas:
                kandidati.add(slovicko)
    return kandidati

def mozny_predek(predchudce):
    if predchudce in all_lemmas:
        return predchudce
    else:
        if predchudce[-1] not in "aáeěiíoóuůúyý":
            dalsi = test(predchudce)
            if len(dalsi) > 1:
                return nejlepsi(dalsi)
            elif len(dalsi) == 1:
                return dalsi.pop()
            else:
                return None

def vytisknout(seznam, bez):
    with open("test9.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
        f.write("PODSTATNÁ JMÉNA ŽENSKÉHO RODU KONČÍCÍ NA SPECIÁLNÍ PŘÍPADY 'KA' A JSOU BEZ PŘEDKA \n")
        f.write("\n")
        f.write("PODSTATNÁ JMÉNA, KE KTERÝM BYL NALEZEN PŘEDEK \n")
        f.write(f"{'PODSTATNÉ JMÉNO'.ljust(20)}{'PŘEDEK'.ljust(20)}\n")
        for i in seznam:
            f.write(f"{i[0].ljust(20)}{i[1].ljust(20)}\n")

        f.write("\n")
        f.write("PODSTATNÁ JMÉNA, PRO NĚŽ NEBYL NALEZEN PŘEDCHŮDCE\n")
        for i in bez:
            f.write(f"{i} \n")



seznam = []
bez = []

def hledani_predka(predek, dite):
    kandidati = {lemma for lemma in all_lemmas if predek in lemma and len(predek) < len(lemma) and lemma != dite and len(dite) > len(lemma)}
    print("ha")
    vysledny = nejlepsi(kandidati)
    return vysledny

for lexeme in lexicon.iter_lexemes():
    if lexeme.pos == "NOUN" and lexeme.lemma.endswith("ka"):
        if lexeme.all_parents == []:
            if 'Gender' in lexeme.feats and lexeme.feats['Gender'] == 'Fem':
                predchudce = lexeme.lemma[:-2]
                hotovo = False
                #veci jako profesorka a profesor
                #ale ne veci jako matka a mat
                if predchudce in all_lemmas:
                    slovo = lexicon.get_lexemes(predchudce)[0]
                    casto_pred = slovo.misc['corpus_stats']['absolute_count']
                    casto_po = lexeme.misc['corpus_stats']['absolute_count']

                    if casto_po < casto_pred:
                        hotovo = True
                        seznam.append((lexeme.lemma, slovo.lemma))
                
                if not hotovo:

                    if lexeme.lemma.endswith(("vka", "čka", "nka")):
                        predchudce = lexeme.lemma[:-4]
                        vysledek = mozny_predek(predchudce)
                        if vysledek is not None:
                            seznam.append((lexeme.lemma, predchudce))
                            hotovo = True

                        else:
                            predchudce = lexeme.lemma[:-2]
                            vysledek = mozny_predek(predchudce)
                            if vysledek is not None:
                                seznam.append((lexeme.lemma, slovo.lemma))
                                hotovo = True
                        
                        if not hotovo:
                            predchudce = lexeme.lemma[:-4]
                            cil = hledani_predka(predchudce, lexeme.lemma)
                            if cil is not None:
                                seznam.append((lexeme.lemma, cil))
                                hotovo = True

                        if not hotovo:
                            bez.append(lexeme.lemma)
                    

                    #pedagožka a pedagog
                    elif lexeme.lemma.endswith("žka"):
                        predchudce = predchudce + "g"

                        if predchudce in all_lemmas:
                            seznam.append((lexeme.lemma, predchudce))
                        else:
                            bez.append(lexeme.lemma)
                        
                    else: #teď pořešit skandinavistka a skandinavista
                        vysledek = mozny_predek(predchudce)
                        if vysledek is not None:
                            seznam.append((lexeme.lemma, predchudce))
    
vytisknout(seznam, bez)

ha


test 10: podstatné jméno končící na 'ost' by mělo být k něčemu připojeno (jasnost, lenost, prázdnost)

In [ ]:
def hledani(slovo):
    rodic = slovo + 'ý'
    dalsi = slovo + 'í'

    if rodic in all_lemmas:
        return rodic
    elif dalsi in all_lemmas:
        return dalsi
    else:
        return None
    
def tisknuti(seznam, osamocen):
    with open("test10.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
        f.write("PODSTATNÁ JMÉNA KONČÍCÍ NA 'OST' A JSOU BEZ PŘEDKA \n")
        f.write("PODSTATNÉ JMÉNO A K NĚMU NALEZENÝ PŘEDEK\n")
        
        f.write(f"{'PODSTATNÉ JMÉNO'.ljust(20)}{'PŘEDEK'.ljust(20)}\n")
        for i in seznam:
            f.write(f"{i[0].ljust(20)}{i[1].ljust(20)}\n")

        f.write("\n")
        f.write("PODSTATNÁ JMÉNA, PRO NĚŽ NEBYL NALEZEN PŘEDCHŮDCE\n")
        f.write("SEŘAZENO PODLE ABSOLUTNÍHO POČTU Z lexeme.misc \n")
        for slovo, predek, vzdalenost in sorted(osamocen, key=lambda x: x[1]):
            f.write(f"{slovo.ljust(20)}{predek.ljust(20)}{str(vzdalenost).ljust(20)}\n")
        
        
    
seznam = []
bez = []
for lexeme in lexicon.iter_lexemes():
    if lexeme.lemma.endswith("ost") and lexeme.all_parents == []:
        if len(lexeme.lemma[:-3]) > 3:
        #nejdřív se podíváme, jestli má rodiče, pokud ne, tak je to problém 
            predchudce = lexeme.lemma[:-3]
            vysledek = hledani(predchudce)

            if vysledek is not None:
                seznam.append((lexeme.lemma, vysledek))
            else:
                predchudce = lexeme.lemma[:-4]
                vysledek = hledani(predchudce)

                if vysledek is not None: 
                    seznam.append((lexeme.lemma, vysledek))
                else:
                    hodnota = lexeme.misc.get("corpus_stats", {}).get("absolute_count", 0)
                    bez.append((lexeme.lemma, hodnota))

tisknuti(seznam, bez)


test 11: stejně dlouhá slova, ale jiný začátek slova

In [ ]:
def alternativy(otec, syn):
    if syn == "ú" + otec[2:] or syn == "o" + otec[2:]:
        return True
    else:
        return None

def levenhstein(prvni, druhy):
    n = len(prvni)
    m = len(druhy)
    tabulka = [[0] * (m + 2) for _ in range(n + 2)]
    for i in range(1, n+ 2):
        tabulka[i][m+1] = n - i + 1
    for j in range(1, m + 2):
        tabulka[n+ 1][j] = m - j + 1
    for i in range(n, 0, -1):
        for j in range(m, 0, -1):
            delta = 1
            if prvni[i - 1] == druhy[j-1]:
                delta = 0
            tabulka[i][j] = min(delta + tabulka[i + 1][j + 1], 1 + tabulka[i + 1][j], 1 + tabulka[i][j + 1])

    return tabulka[1][1]

def tisk(resitelne, neresitelne):
    with open("test11.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
        f.write("SLOVA, KTERÁ JSOU PODOBNĚ KRÁTKÁ JAKO JEJICH PŘEDEK, ALE ZAČÍNAJÍ NA JINÉ PÍSMENO \n")
        f.write("SEZNAM TĚCH, KTERÉ BY MĚLY BÝT PROHOZENÉ, TEDY RODIČ BY MĚL BÝT POD SYNEM\n")
        f.write(f"{'PŮVODNÍ SLOVO'.ljust(20)}{'PŮVODNÍ PŘEDEK'.ljust(20)}\n")
        f.write("\n")
        for i in resitelne: 
            f.write(f"{i[0].ljust(20)}{i[1].ljust(20)}\n")
        f.write("\n")

        f.write("SEZNAM TĚCH, KTERÉ SPLNILY KRITÉRIA, SEŘAZENY PODLE LEVENHSTEINOVY VZDÁLENOSTI \n")
        f.write(f"{'PŮVODNÍ SLOVO'.ljust(20)}{'PŮVODNÍ PŘEDEK'.ljust(20)}{'VZDÁLENOST'.ljust(20)}\n")
        for i in  sorted(neresitelne, key=lambda x: x[2], reverse=True):
            f.write(f"{i[0].ljust(20)}{i[1].ljust(20)}{str(i[2]).ljust(20)}\n")
    
            
seznam = []
mozne = []

for lexeme in lexicon.iter_lexemes():
    for i in lexeme.all_parents:
        if len(lexeme.lemma) < len(i.lemma) + 2 and lexeme.lemma[0] != i.lemma[0]:
            #pojistit rozdily:
            if lexeme.lemma[0].lower() == i.lemma[0].lower(): #čáslavský x Čáslav
                continue
            elif i.lemma.startswith("-"):
                continue
            elif len(lexeme.lemma) < 7 and levenhstein(lexeme.lemma, i.lemma) < 2:
                continue
            elif len(lexeme.lemma) > 6 and levenhstein(lexeme.lemma, i.lemma) < 3:
                continue
            elif "apgrejd" in lexeme.lemma:
                if "upgrade" + lexeme.lemma[7:] == i.lemma:
                    continue  
            else:
                reseni = alternativy(lexeme.lemma, i.lemma)
                if reseni is None:
                    seznam.append((lexeme.lemma, i.lemma, levenhstein(lexeme.lemma, i.lemma)))
                else:
                    mozne.append((lexeme.lemma, i.lemma))

tisk(mozne, seznam)


test 12: test na slova končící na 'á'

In [ ]:
seznam = []
bez = []

def tisk(seznam, bez):   
    with open("test12.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
        f.write("SLOVA KONČÍCÍ NA 'Á, KTERÁ NEJSOU K NIČEMU PŘIPOJENA")
        f.write("SEZNAM TĚCH, KE KTERÝM BYL NALEZEN POTENCIÁLNÍ PŘEDEK\n")
        f.write(f"{'NALEZENÉ SLOVO'.ljust(20)}{'MOŽNÝ PŘEDEK'.ljust(20)}\n")
        f.write("\n")
        for i in seznam: 
            f.write(f"{i[0].ljust(20)}{i[1].ljust(20)}\n")
        f.write("\n")

        f.write("SEZNAM TĚCH, KE KTERÝM PŘEDEK NEBYL NALEZEN \n")
        f.write("NALEZENÉ SLOVO \n")
        for i in bez:
            f.write(f"{i}\n")
        
all_lemmas1 = {lemma.lower() for lemma in all_lemmas}

for lexeme in lexicon.iter_lexemes():
    if lexeme.lemma[-1] in "á":
        if lexeme.all_parents == [] and (len(lexeme.lemma) > 3 or lexeme.lemma[0].isupper()):
            rodic = ""
            dalsi = ""
            if lexeme.lemma.endswith("ová"): #and lexeme.lemma != "Nová":
                if lexeme.lemma[0].isupper():
                    rodic = lexeme.lemma[:-3]
                    if rodic[-2] not in "aáeěiíoóuůúyý":
                        pomoc = rodic[:-1] + "e" + rodic[-1]
                        dalsi = pomoc
                else:
                    rodic = lexeme.lemma[:-1] + "ý" 

            elif lexeme.lemma.endswith("ická"):
                rodic = lexeme.lemma[:-2] + "e"
            else:
                rodic = lexeme.lemma[:-1].lower()
                rodic = rodic + "ý"

            if rodic in all_lemmas:
                seznam.append((lexeme.lemma, rodic))
            elif dalsi in all_lemmas:
                seznam.append((lexeme.lemma, dalsi))

            else:
                if lexeme.lemma.endswith("ová"):
                    if rodic + "a" in all_lemmas:
                        seznam.append((lexeme.lemma, rodic + "a"))
                    else:
                        rodic = rodic.lower()
                        if rodic in all_lemmas1:
                            seznam.append((lexeme.lemma, rodic))
                        elif dalsi in all_lemmas1:
                            seznam.append((lexeme.lemma, dalsi))
                        else:
                            if lexeme.lemma.endswith("ová"):
                                if rodic + "a" in all_lemmas1:
                                    seznam.append((lexeme.lemma, rodic + "a"))
                                else:
                                    bez.append(lexeme.lemma)
tisk(seznam, bez)

test 15: když geografický pojem začínající velkým písmenem má potomka s malým písmenem (Absurdistán -> absurdistán)

In [25]:
#absurdistán by měl být odvozen od Absurdistánu?? nebo nějak naopak 
def tisk(seznam):
    with open("test15.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
        f.write("GEOGRAFICKÉ POJMY, KTERÉ MAJÍ IDENTICKÉ SLOVO ZAČÍNAJÍCÍ MALÝM PÍSMENEM A NEJSOU K SOBĚ PŘIPOJENI \n")
        f.write(f"{'GEOGRAFICKÝ POJEM'.ljust(20)}{'ODVOZENÉ SLOVO'.ljust(20)}\n")
        f.write("\n")
        for i in seznam: 
            f.write(f"{i[0].ljust(20)}{i[1].ljust(20)}\n")

seznam = []

for lexeme in lexicon.iter_lexemes():
    if 'NameType' in lexeme.feats and lexeme.feats['NameType'] == 'Geo' and lexeme.lemma[0].isupper():
        tvar = lexeme.lemma.lower()
        if tvar in all_lemmas:
            slovo = lexicon.get_lexemes(tvar)[0]
            if lexeme not in slovo.all_parents and slovo not in lexeme.all_parents:
                seznam.append((lexeme.lemma, slovo.lemma))
                

tisk(seznam)


In [28]:
pred, po = "", ""

for lexeme in lexicon.iter_lexemes():
    if lexeme.lemma == "masochista":
        for i in lexeme.all_parents:
            print(i.lemma)
        print("ha")


masochizmus
ha


test 16: když je dítě kratší než rodič (tohle je test kvůli dvěma slovům)

In [ ]:
def tisk(seznam):
    with open("test16.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
        f.write("SLOVA, JEJICHŽ RODIČE JSOU KRATŠÍ NEŽ ONY SAMY\n")
        f.write("SEŘAZENO PODLE LEVENHSTEINOVY VZDÁLENOSTI \n")
        f.write("\n")
        f.write(f"{'ZKOUMANÉ SLOVO'.ljust(20)}{'RODIČ'.ljust(20)}{'VZDÁLENOST'.ljust(20)}\n")
        f.write("\n")
        for i in sorted(seznam,  key=lambda x: x[2], reverse=True): 
            f.write(f"{i[0].ljust(20)}{i[1].ljust(20)}{str(i[2]).ljust(20)}\n")

seznam = []
for lexeme in lexicon.iter_lexemes():
    for i in lexeme.all_parents:
        if len(i.lemma) > len(lexeme.lemma) + 2 and lexeme.pos == i.pos:
            casto_i = i.misc["corpus_stats"]["absolute_count"]
            casto_lemma = lexeme.misc["corpus_stats"]["absolute_count"]

            if casto_i < casto_lemma:
                if lexeme.lemma.endswith(("ismus", "izmus")) and levenhstein(lexeme.lemma[:-5], i.lemma) < 2:
                    continue
                elif lexeme.lemma.endswith(("ika")) and levenhstein(lexeme.lemma[:-3], i.lemma) < 2:
                    continue
                else:
                    seznam.append((lexeme.lemma, i.lemma, levenhstein(lexeme.lemma, i.lemma)))

tisk(seznam)

test 17: aforizmus -> aforista a aforismus                                                       

In [31]:
def tisk(ismy, jenom_jeden):
    with open("test17.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre¨
        f.write("TEST ZABÝVAJÍCÍ SE VZTAHY MEZI -IZMUS, -ISMUS A -ISTICKÝ\n")
        f.write("SEZNAM SLOV, KTERÉ KONČÍ NA 'IZMUS', ALE NEJSOU PŘIPOJENA K VERZI 'ISMUS'\n")
        f.write(f"{'-IZMUS'.ljust(30)}{'-ISMUS'.ljust(30)}\n")
        f.write("\n")
        for i in ismy: 
            f.write(f"{i[0].ljust(30)}{i[1].ljust(30)}\n")
        f.write("\n")

        f.write("SEZNAM SLOV KONČÍCÍ NA 'ISTICKÝ', KTERÁ JSOU PŘIPOJENA NA 'IZMUS', ALE NE 'ISMUS\n")
        f.write(f"{'-ISTICKÝ'.ljust(30)}{'ŠPATNĚ: -IZMUS'.ljust(30)}{'SPRÁVNĚ: -ISMUS'.ljust(30)}\n")
        f.write("\n")
        for i in jenom_jeden: 
            f.write(f"{i[1].ljust(30)}{i[0].ljust(30)}{i[2].ljust(30)}\n")
        f.write("\n")

    
seznam = [] #když masochizmus není příbuzný masochismu
nespojeno = [] #nespojeno - nacionalista je pouze pod nacionalizmem a ne pod nacionalismem
#spojeno_blbe - pripojen k obema (nacionalista jak pod nacionalizmus tak nacionalismus)

all_lemmas = {lex.lemma for lex in lexicon.iter_lexemes()}

for lexeme in lexicon.iter_lexemes():
    if lexeme.lemma.endswith("izmus"): #podívám se, jestli má předka to slovo se 's'
        rodic = lexeme.lemma[:-4] + "s" + lexeme.lemma[-3:]
        if lexeme.all_parents == []:
            if rodic in all_lemmas:
                seznam.append((lexeme.lemma, rodic))
                #f.write(f"????{lexeme.lemma} a {rodic} by měly být příbuzní \n")
        
        #pak se podívám do všech dětí, jestli tam nemá 'ista', i když by tam být neměl, protože to je od jiného slova
        deti = lexeme.children 
        for d in deti:
            if d.lemma.endswith("ista"):
                if rodic in all_lemmas: 
                    #podívám se na všechny předky toho dítěte od izmu
                    predci = d.all_parents
                    jmena = []
                    for i in predci: 
                        jmena.append(i.lemma)
                    
                    #pokud tam není -ismus, tak je to špatně
                    if rodic not in jmena: #pokud má to dítě dva rodiče - masochista má jak masochizmus a masochismus
                        nespojeno.append((lexeme.lemma, d.lemma, rodic))
                

tisk(seznam, nespojeno)


test 18: gothajský/gotajský a ethiopský a etiopský

In [39]:
def lezeme_nahoru(dite, hledany): #podívám se, jestli nejsou nějak propojené
    rodice = dite.all_parents
    if not rodice:
        return False  # nedošli jsme k hledanému
    if hledany in rodice:
        return True  # hledaný je přímo rodičem
    for r in rodice:
        if lezeme_nahoru(r, hledany):  # rekurzivně jdeme dál
            return True
    return False  # pokud jsme ho nikdy nenašli

#chci zjistit, jestli jsou ta slova k sobě nějak navázaná, i když ne napřimo
def jsou_pribuzni(bez_h, h):
    otec_h = lexicon.get_lexemes(h)[0]
    rodice_h = otec_h.all_parents
    otec_bez = lexicon.get_lexemes(bez_h)[0]
    rodice_bez_h = otec_bez.all_parents

    #jestli je gotaj otcem gothaje
    if any(parent.lemma == bez_h for parent in rodice_h):
        return 0
    #jestli je gothaj otcem gotaje
    elif any(parent.lemma == h for parent in rodice_bez_h):
        return 1
    else:
        #pokud to není ani jedno, tak se podíváme, jestli nejsou příbuzní oklikou
        #je gotaj nějak dítětem gothaje?
        etiopie = lezeme_nahoru(otec_bez, otec_h)
        #je gothaj nějak dítětem gotaje?
        gothaj = lezeme_nahoru(otec_h, otec_bez)

        if etiopie or gothaj:
            return 2
        else:
            return 3
        
def pomery(celkove, h, bez_h, vzdalene, vubec, dohromady):
    seznam = { "h": h/celkove*100, "bez_h" : bez_h/celkove*100, "vubec" : vubec/celkove*100,  "vzdalene" : vzdalene/celkove*100}
    vypis = {"h" : "rodičem v tomto vztahu je slovo obsahující 'h' navíc", "bez_h":"rodičem v tom vztahu je slovo neobsahující zkoumané 'h'", 
             "vubec": "tato dvojice slov není nijak příbuzná", "vzdalene": "tato dvojice slov je příbuzná, ale ne napřímo"}

    #chci nejdriv vypsat všechny četnosti a poté je vypsat jejich pořadí  
    with open("test18.txt", "w", encoding="utf-8") as f:
        f.write("S JAKOU ČETNOSTÍ SE V DERINETU NACHÁZÍ SPOJENÍ SLOV, KTERÉ SE LIŠÍM JEDNÍM H \n")
        f.write("TABULKA ČETNOSTÍ OD NEJČASTĚJŠÍHO VZTAHU DO NEJMÉNĚ ČASTÉHO: \n")
        sortovano = sorted(seznam.items(), key=lambda x: x[1], reverse=True)
        f.write(f"{'TYP VZTAHU'.ljust(70)}{'ČETNOST V PROCENTECH'.ljust(30)}\n")
        for k, v in sortovano:
            f.write(f"{vypis[k].ljust(70)}{str(v).ljust(30)} \n")

        for k, v in sortovano: #kdyz tam dodam with, tak se mi to samo zavre
            f.write("\n")
            f.write(f"{vypis[k].upper()} \n")
            if k == "h" or k == "bez_h":
                f.write(f"  {'OTEC'.ljust(20)}{'SYN'.ljust(20)}\n")
            seznamik = dohromady[k]
            for i in sorted(seznamik, key=lambda x: x[0]):
                f.write(f"      {i[0].ljust(20)}{i[1].ljust(20)}\n")

celkove = 0

#jestli je jeden z nich otcem druhého, jestli jsou nějak příbuzní a jestli nejsou vůbec příbuzní
h_ne, h_ano, vzdalene, vubec = 0, 0, 0, 0 
h_ne_seznam, h_ano_seznam, vzdalene_seznam, vubec_seznam = set(), set(), set(), set()

for lexeme in lexicon.iter_lexemes():
    if "h" in lexeme.lemma and not lexeme.lemma.startswith("-"):

        #poodstraňuju všechna h, která se v tom slově nachází
        indexiky = []
        for i in range(len(lexeme.lemma)):
            if lexeme.lemma[i] == "h":
                indexiky.append(i)

        novotvary = []
        for j in indexiky:
            novotvary.append(lexeme.lemma[:j] + lexeme.lemma[j+1:])

        for k in novotvary:
            if k in all_lemmas:
                #print(k)
                stav = jsou_pribuzni(k, lexeme.lemma)
                celkove += 1

                if stav == 0: #znamená to, že etiopský je rodičem ethiopský
                    h_ano += 1
                    h_ano_seznam.add((k, lexeme.lemma))
                elif stav == 1: #gothaj je rodičem gotaj
                    h_ne += 1 
                    #je to vždycky (rodič, syn)
                    h_ne_seznam.add((lexeme.lemma, k))
                elif stav == 2:
                    vzdalene += 1
                    vzdalene_seznam.add((lexeme.lemma, k))
                else:
                    vubec += 1
                    vubec_seznam.add((lexeme.lemma, k))

dohromady = {"h": h_ano_seznam, "bez_h" : h_ne_seznam, "vubec" : vubec_seznam,  "vzdalene" : vzdalene_seznam}
pomery(celkove, h_ano, h_ne, vzdalene, vubec, dohromady)

test 19: jeden rodič, ale rozdíl mezi velikostí rodiče a dítěte je veliká 

In [ ]:
#tady dohledáváme slova, která by se tam měla nacházet, ale nenachází se tam 

with open("test19.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
    for lexeme in lexicon.iter_lexemes():
        if len(lexeme.lemma) >= 10 and len(lexeme.all_parents) == 1 and len(lexeme.parent.lemma) <= 5:
            if not lexeme.lemma.endswith(("ovský", "owský", "ovat", "ový", "vitý")):
                rodic = lexeme.parent.lemma
                index = lexeme.lemma.find(rodic)
                if index != -1:
                    retezec = lexeme.lemma[:index]
                    if retezec in all_lemmas:
                        #seradit to podle delky retezce
                        f.write(f"{lexeme.lemma}, {lexeme.parent.lemma}, CHYBÍ {retezec} \n")


test 20: prefixy a hranice slov

In [ ]:
#dodat tam ty hranice slov

def tvorba_pruniku(prvni, druhy): #prvni je kratsi
    tabulka = [[0 for _ in range(len(druhy))] for _ in range(len(prvni))]
    
    for i in range(len(prvni)): #ted mam kdy jsou to stejne hodnoty
        for j in range(len(druhy)):
            if prvni[i] == druhy[j]:
                tabulka[i][j] = 1
    
    slovicko = ""
    slovo = ""
    vysledek = []
    for i in range(len(prvni)):
        for j in range(len(druhy)):
            slovicko = ""
            while i < len(prvni) and j < len(druhy) and tabulka[i][j] == 1:
                slovicko += prvni[i]
                i += 1
                j += 1
            if len(slovo) < len(slovicko):
                slovo = slovicko
                vysledek = [slovicko]
            elif len(slovo) == len(slovicko):
                vysledek.append(slovicko)

    return vysledek
    

def hledani_pruniku(seznam):
    prubezny = [seznam[0]]
    for i in range(len(seznam)):
        if i == 0:
            continue
        else: 
            pomocnik = []
            for j in prubezny:
                pomocnik.extend(tvorba_pruniku(j, seznam[i]))
            prubezny = pomocnik    

    return sorted(prubezny, key=len, reverse=True)

def deti(f, deticky, podstring):
    mozne_deti = []
    for i in all_lemmas:
        if podstring in i and i not in deticky:
            mozne_deti.append(i)

    f.write(f"{lexeme.lemma}\n")
    for i in mozne_deti:
        f.write(f"      {i}\n")

with open("test20.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
    for lexeme in lexicon.iter_lexemes():
        if lexeme.pos == "Affixoid" or lexeme.pos == "NeoCon":
            deticky = []
            print(lexeme.lemma)
            for i in lexeme.children:
                deticky.append(i.lemma)

            if deticky == [] :
                aff = lexeme.lemma[1:-1]
                deti(f, deticky,aff)
            
            elif len(deticky) == 1:
                aff = lexeme.lemma[1:-1]
                if aff in deticky[0]:
                    deti(f, deticky, aff)
                
                #kdyz se bude lisit rodic a dite, tak nevím podle čeho se řídit

            else: 
                #tuhle prisernou funkci pisu kvuli tomu, že název affixoidu neodpovídá tomu, co se nachází ve slovech
                podstringy = hledani_pruniku(deticky)
                podstring = podstringy[0]

                deti(f, deticky, podstring)           

-acid-
-aden-
-aer-
-aero-
-agog-
-agora-
-agro-
-aichm-
-al-
-alg-
-ana-
-andr-
-angi-
-angl-
-anter-
-anthrop-
-antrop-
-arachn-
-arch-
-arche-
-arci-
-arist-
-astr-
-audi-
-aut-
-balne-
-bi-
-bio-
-blank-
-blast-
-brady-
-bronch-
-byr-
-ceci-
-centr-
-cephalo-.-kephalo-
-cervik-
-chol-
-chondr-
-chord-
-chron-
-circum-
-coel-
-cyan-
-cykl-
-cyklo-
-cyt-
-daktyl-
-dein-
-dem-
-dendr-
-dermat-
-di-
-dia-
-doch-
-dol-
-dys-
-e-
-ek-
-eko-
-ekt-
-elektr-
-en-
-enanti-
-end-
-ent-
-enter-
-ereuth-
-erg-
-etho-
-ethyl-
-etn-
-etno-
-eur-
-euro-
-ex-
-exo-
-fac-
-fag-
-felin-
-ferr-
-fid-
-fil-
-flori-
-fob-
-fon-
-fot-
-frank-
-free-
-fug-
-fyt-
-galvan-
-gam-
-gastr-
-ge-
-gen-
-geo-
-geront-
-gestalt-
-giga-
-gnos-
-gon-
-graf-
-gram-
-granul-
-gymn-
-gyn-
-hepat-
-heter-
-hod-
-hol-
-hom-
-hydr-
-hydroxy-
-hygr-
-hymen-
-hyper-
-hypn-
-hypo-
-idio-
-infra-
-inter-
-intra-
-iso-
-kalyptr-
-kancer-
-karcin-
-kardi-
-karp-
-kilo-
-kinet-
-klast-
-klin-
-kok-
-kortik-
-krat-
-krimi-
-kry-


test 21: Hubbert a Hubert a podobné 

In [ ]:
with open("test21.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
    #tady najdu to, kdyz se ta pismena budou opakovat
    for lexeme in lexicon.iter_lexemes():
        if any(lexeme.lemma[i] == lexeme.lemma[i + 1] for i in range(len(lexeme.lemma) - 1)):
            #musim najit ten index
            for i in range(lexeme.lemma) - 1:
                if lexeme.lemma[i] == lexeme.lemma[i + 1]:
                    nove = lexeme.lemma[:i] + lexeme.lemma[i+1:]
                    if nove in all_lemmas:
                        #potom uz je to znovu h a bez h

test 22: pesimismus a pesimizmus, ale něco jako autizta a austista

In [ ]:
with open("test22.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
    for lexeme in lexicon.iter_lexemes():
        if "z" in lexeme.lemma: #v korpusech se častěji nachází s než z
            for i in range(len(lexeme.lemma)):
                if lexeme.lemma[i] == "z":
                    nove = lexeme.lemma[:i] + "z" + lexeme.lemma[i+1:]
                    if nove in all_lemmas: 
                        # tak znovu h a bez h


test 23: acyl -> vezmu všechno s acylem, vezmu jeho další části a poté se podívám v jakých dalších slovech se nacházejí

In [ ]:
def hledani_podslov(slovo, uvnitr, videne, f):
    index = slovo.index(uvnitr)
    pred, po = "", ""
    vsechno_po, vsechno_pred = set(), set()
    if index != 0:
        pred = lexeme.lemma[:index]
    po = lexeme.lemma[index + len(slovo):]

    if pred != "" and pred not in videne:
        vsechno_pred = {lemma for lemma in all_lemmas if pred in lemma}
        videne.add(pred)
    if po not in videne:
        vsechno_po = {lemma for lemma in all_lemmas if pred in lemma}
        videne.add(po)

    celkove = vsechno_po | vsechno_pred

    for i in celkove:
        slova = lexicon.get_lexemes(i)
        if slova == []:
            continue
        else:
            slovo = slova[0]
            rodice = slovo.all_parents
            if i not in rodice:
                f.write(f"{pred}, {slovo}  \n")
    
    for i in celkove:
        if i not in videne:
            hledani_podslov(i, videne, f)


with open("test23.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
    videne = set()
    seznam = []
    for lexeme in lexicon.iter_lexemes():
        if "acyl" in lexeme.lemma and lexeme.feats['Loanword'] and lexeme.lemma != "acyl":
           seznam.append(lexeme.lemma)
        
    for i in seznam:
        hledani_podslov("acyl", videne, f)

ValueError: substring not found

test 24: Bukowský, bukovský a celkově tohle to

In [86]:
with open("test24.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
    for lexeme in lexicon.iter_lexemes():
        if lexeme.lemma.endswith("owský"):
                rodice = []
                for i in lexeme.all_parents:
                    rodice.append(i.lemma)
            
                nove = lexeme.lemma[:-4] + "v" + lexeme.lemma[-3:]
                if "w" in nove:
                    index = nove.index("w")
                    lepsi = nove[:index] + "v" + nove[index + 1:]
                    if lepsi in all_lemmas and lepsi not in rodice:
                        f.write(f"{lexeme.lemma}, {lepsi} \n")
                    elif lepsi.lower() in all_lemmas and lepsi.lower() not in rodice:
                        f.write(f"{lexeme.lemma}, {lepsi.lower} \n")

                else:
                    if nove in all_lemmas and nove not in rodice:
                        f.write(f"{lexeme.lemma}, {nove} \n")
                    elif nove.lower() in all_lemmas and nove.lower() not in rodice:
                        f.write(f"{lexeme.lemma}, {nove.lower()} \n")


test 25: fake affixoidy

u nějakých affixoidů a neoconů jsou připojena slova, která by tam podle mě vůbec neměla být 
příklad: -takt- -> architekt nebo -pneu- ->  tachypnoe

In [ ]:
with open("fake_affixoid.txt", "w", encoding="utf-8") as f: #kdyz tam dodam with, tak se mi to samo zavre
    for lexeme in lexicon.iter_lexemes():
        if (lexeme.pos == "Affixoid" or lexeme.pos == "NeoCon"):  #and len(lexeme.lemma) > 4:
            # je to 4, protoze -my- jeste obsahuje ty dve pomlcky
            #kdyz to bude delsi nez dva, zbavim se tech casti slov jako jsou -my- -e-
            slovo = lexeme.lemma[1:-1] #tak bych se mela zbavit tech pomlcek

            for lem in lexeme.children:
                if slovo not in lem.lemma or not lem.lemma.startswith(slovo):
                    f.write(f"{lem.lemma}, {slovo} \n")
